# Computational Theory 

## Introduction

This repository demonstrates a complete implementation of the SHA-256 cryptographic hash algorithm, built from first principles. The work showcases both theoretical understanding and practical implementation skills relevant to modern cybersecurity and cryptographic applications.

**Project Structure:**

- **Problem 1**: Implementation of SHA-256's core bitwise functions ([`parity`](#parity-function), [`ch`](#ch-choose-function), [`maj`](#majority-function), and [sigma operations](#sigma0-function))
- **Problem 2**: Mathematical derivation of SHA-256 constants from cube roots of prime numbers ([`primes`](#the-primesn-function))
- **Problem 3**: Message preprocessing including padding and block parsing ([`block_parse`](#problem-3-padding), [`print_blocks`](#helper-function))
- **Problem 4**: Complete SHA-256 compression function ([`hash`](#the-hash-function))
- **Problem 5**: Password security analysis and dictionary attacks ([`find_password`](#passwords))

<!-- The implementation prioritises clarity, with extensive comments explaining both the cryptographic theory and Python implementation details throughout each problem section. -->

## Libraries

In [362]:
# For numerical data and methods 
import numpy as np 
# For symbolic mathematics
import sympy as sp

## Problem 1: Binary Words and Operations

This section implements the fundamental cryptographic functions that serve as the building blocks for SHA-256 hash computation. These functions operate on 32-bit words and provide the essential bitwise operations required for the SHA-256 compression algorithm.

**Basic Boolean Functions:**
- **`parity(x,y,z)`** - XOR operation for error detection and bit diffusion
- **`ch(x,y,z)`** - Choose function implementing conditional bit selection
- **`maj(x,y,z)`** - Majority function for fault tolerance and consensus

**Bit Manipulation Operations:**
- **`rotr(x,n)`** - Rotate right operation for cryptographic mixing

**SHA-256 Compression Functions:**
- **`sigma_upper_0/1(x)`** - Uppercase sigma functions for working variable transformation
- **`sigma_lower_0/1(x)`** - Lowercase sigma functions for message schedule expansion

These functions work together to provide the mathematical foundation for SHA-256's cryptographic security, implementing the core operations that transform input data through multiple rounds of secure hashing.


The `numpy.uint32()` constructor ([see official documentation](https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.int32)) ensures that values are stored and treated as 32-bit unsigned integers in NumPy, which is important for consistency and compatibility in numerical computations.


### Parity Function

$\mathrm{Parity}(x,y,z)=x\oplus y\oplus z$

The `parity` function calculates the bitwise XOR of three 32-bit integers, as defined in the SHA-1 algorithm. For each bit position, it returns a result of 1 when an odd number of the input values is set to 1, and 0 otherwise. This is implemented as `x ^ y ^ z`.

Parity originates from the need for simple error detection in data transmission. The parity bit can detect such errors by identifying mismatches in the transmission and by checking if the number of received data bits matches the expected parity ([see source](https://www.techtarget.com/searchstorage/definition/parity)). If there is no match, an error is generated.
- For example
    - If the sender used even parity and the sequence 1011 arrives (three 1s), the mismatch signals an error.

Parity's purpose is to enable the receiver to detect errors that may have occurred during transmission.

In [363]:
def parity(x,y,z):
    """Calculate the parity of three 32-bit integers"""
    # np.uint32 ensures inputs are treated as 32-bit unsigned integers as per SHA-1 specification.
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    # Mask with 0xFFFFFFFF to keep only the lower 32 bits, handles negative numbers correctly and simulates 32-bit overflow behavior.
    x = np.uint32(x & 0xFFFFFFFF)
    y = np.uint32(y & 0xFFFFFFFF)
    z = np.uint32(z & 0xFFFFFFFF)

    # Calculate the bitwise XOR of the three 32-bit values to get the parity.
    parity_output = np.uint32(x ^ y ^ z)
    # Return the result.
    return parity_output

### Test Parity Function
This section tests the `parity` function with various inputs to verify its correctness.

In [364]:
# This array holds the test cases for the parity function - each test case is a tuple containing: the test label, the input arguments, and the expected result.
tests = [
    ("TEST 1: parity(0, 0, 0)", (0, 0, 0), 0), # Test 1: All zeros.
    ("\nTEST 2: parity(1, 0, 0)", (1, 0, 0), 1), # Test 2: One non-zero input.
    ("\nTEST 3: parity(1, 1, 0)", (1, 1, 0), 0), # 3: Two non-zero inputs.
    ("\nTEST 4: parity(1, 1, 1)", (1, 1, 1), 1), # 6: All ones.
]
# look through and run each test
for label, args, expected_result in tests:
    print(label)
    test_result = parity(*args)
    print("Result :", (int(test_result)), "Expected :", (expected_result), "Correct :", int(test_result) == expected_result)


TEST 1: parity(0, 0, 0)
Result : 0 Expected : 0 Correct : True

TEST 2: parity(1, 0, 0)
Result : 1 Expected : 1 Correct : True

TEST 3: parity(1, 1, 0)
Result : 0 Expected : 0 Correct : True

TEST 4: parity(1, 1, 1)
Result : 1 Expected : 1 Correct : True


### Ch (choose) Function
$\mathrm{Ch}(x,y,z) = (x \wedge y)\ \oplus\ (\neg x \wedge z)$

The `choose` function calculates a conditional selection among three 32-bit integers, as used in the SHA-224 and SHA-256 algorithms. For each bit position, it returns the bit from `y` if the bit in `x` is 1, or the bit from `z` if the bit in `x` is 0. This is implemented as `(x & y) ^ (~x & z)`.

The choose function originates from the need for **conditional logic in digital circuits**, implementing a bitwise multiplexer operation that selects between two inputs based on a control bit ([see source](https://en.wikipedia.org/wiki/Multiplexer)).

- For example
    - If `x = 1`, the function selects bits from `y` (choose y when x is true)
    - If `x = 0`, the function selects bits from `z` (choose z when x is false)
    - Outputs: `Ch(1,1,0) = 1` and `Ch(0,1,0) = 0`


In [365]:
def ch(x, y, z):
    """Calculate the choose function for three 32-bit integers"""
    # np.uint32 ensures inputs are treated as 32-bit unsigned integers as per SHA-1 specification.
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    # Mask with 0xFFFFFFFF to keep only the lower 32 bits, handles negative numbers correctly and simulates 32-bit overflow behavior.
    x = np.uint32(x & 0xFFFFFFFF)
    y = np.uint32(y & 0xFFFFFFFF)
    z = np.uint32(z & 0xFFFFFFFF)
    
    # Bitwise AND of x and y.
    bitwise_and = (x & y)
    # Bitwise complement of x, then AND with z.
    bitwise_complement_and = (~x) & z
    # Calculate the bitwise XOR of the two results and cast to 32-bit integer.
    # This selects bits from y or z based on the value of x.
    ch_output = np.uint32(bitwise_and ^ bitwise_complement_and)
    
    # Return the result.
    return ch_output 

### Test Ch Function
This section tests the `ch` function with various inputs to verify its correctness.

In [366]:
# This array holds the test cases for the parity function - each test case is a tuple containing: the test label, the input arguments, and the expected result.
tests = [
    ("TEST 1: ch(0, 0, 0)", (0, 0, 0), 0),
    ("\nTEST 2: ch(1, 1, 1)", (1, 1, 1), 1),
    ("\nTEST 3: ch(0, 0, 1)", (0, 0, 1), 1),
    ("\nTEST 4: ch(1, 1, 0)", (1, 1, 0), 1),
    ("\nTEST 5: ch(-1, 0, 1)", (-1, 0, 1), 0),
]

for label, args, expected_result in tests:
    print(label)
    test_result = ch(*args)
    print("Result :", hex(int(test_result)), "Expected :", hex(expected_result), "Correct :", test_result == expected_result)


TEST 1: ch(0, 0, 0)
Result : 0x0 Expected : 0x0 Correct : True

TEST 2: ch(1, 1, 1)
Result : 0x1 Expected : 0x1 Correct : True

TEST 3: ch(0, 0, 1)
Result : 0x1 Expected : 0x1 Correct : True

TEST 4: ch(1, 1, 0)
Result : 0x1 Expected : 0x1 Correct : True

TEST 5: ch(-1, 0, 1)
Result : 0x0 Expected : 0x0 Correct : True


### Majority Function
$\mathrm{Maj}(x,y,z) = (x \wedge y) \oplus (x \wedge z) \oplus (y \wedge z)$

The `maj` function, used in the SHA-224 and SHA-256 algorithms, calculates the majority value among three 32-bit integers. For each bit position, it returns 1 if at least two of the three bits among `x`, `y`, and `z` are 1, and 0 otherwise. This is implemented as `(x & y) ^ (x & z) ^ (y & z)`.

Majority originates from the need for **fault-tolerant decision making** in digital systems. The majority function can detect and correct errors by implementing voting logic where the output follows the majority of inputs ([see source](https://en.wikipedia.org/wiki/Majority_function)). If one input is corrupted, the other two can still determine the correct output.

- For example
    - `Maj(1,1,0) = 1` and `Maj(1,0,0) = 0` (majority of two 1s is 1, majority of two 0s is 0)

Majority's purpose is to enable reliable computation in the presence of potential hardware failures or data corruption.

In [367]:
def maj(x, y, z):
    """Calculate the majority value of three 32-bit integers."""
    # The np.uint32() constructor ensures inputs are treated as unsigned 32-bit integers. 
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    # Mask with 0xFFFFFFFF to ensure only the lower 32 bits are kept to handle negative inputs correctly.
    x = np.uint32(x & 0xFFFFFFFF)
    y = np.uint32(y & 0xFFFFFFFF)
    z = np.uint32(z & 0xFFFFFFFF)

    # Bitwise AND of x and y.
    bitwise_and_x_y = (x & y)   
    # Bitwise AND of x and z.
    bitwise_and_x_z = (x & z) 
    # Bitwise AND of y and z.   
    bitwise_and_y_z = (y & z)   

    # Calculate the bitwise XOR of the three results to get the majority value and cast to a 32-bit integer.
    maj_output = np.uint32(bitwise_and_x_y ^ bitwise_and_x_z ^ bitwise_and_y_z)
    # Return the result.
    return maj_output 

### Test Maj Function
This section tests the `maj` function with various inputs to verify its correctness.

In [368]:
# This array holds the test cases for the maj function - each test case is a tuple containing: the test label, the input arguments, and the expected result.
tests = [
    ("TEST 1: maj(0, 0, 0)", (0, 0, 0), 0), # Test 1: All zeros
    ("\nTEST 2: maj(1, 1, 1)", (1, 1, 1), 1), # Test 2: All ones
    ("\nTEST 3: maj(1, 0, 0)", (1, 0, 0), 0), # Test 3: One non-zero input
]
# look through and run each test
for label, args, expected_result in tests:
    print(label)
    test_result = maj(*args)
    print("Result :", (int(test_result)), "Expected :", (expected_result), "Correct :", int(test_result) == expected_result)


TEST 1: maj(0, 0, 0)
Result : 0 Expected : 0 Correct : True

TEST 2: maj(1, 1, 1)
Result : 1 Expected : 1 Correct : True

TEST 3: maj(1, 0, 0)
Result : 0 Expected : 0 Correct : True


### Rotate Right (rotr)
This helper function performs a bitwise rotate right operation on a 32-bit word. It is used in cryptographic algorithms such as SHA-1 and SHA-224/256, where rotation operations are part of the hash computation. In this project, `rotr` is a key component in the implementation of the sigma functions required for these algorithms.

In [369]:
def rotr(x,n):
    """Rotate right operation for 32-bit words: A helper function for the sigma functions."""
    # The np.uint32() constructor ensures inputs are treated as unsigned 32-bit integers. 
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    x = np.uint32(x)
    # 32 bit word size
    # Rotate right by n bits and ensure result is within 32 bits using masking.
    rotr_output = np.uint32(((x >> n) | (x << np.uint32(32 - n))) & np.uint32(0xFFFFFFFF))

    # Return the output
    return rotr_output


### Sigma0 Function
$\Sigma_0(x) = \text{ROTR}^2(x) \oplus \text{ROTR}^{13}(x) \oplus \text{ROTR}^{22}(x)$

The `sigma0` function applies a series of bitwise operations to a single 32-bit integer, as used in the SHA-224 and SHA-256 algorithms. It computes the bitwise XOR of three right-rotated versions of the input value - one rotated by 2 bits, one by 13 bits, and one by 22 bits. This is implemented as `ROTR^2(x) ^ ROTR^13(x) ^ ROTR^22(x)`.

Sigma0 originates from the need for **complex bit diffusion in cryptographic hash functions**. The specific rotation amounts (2, 13, 22) were carefully chosen to maximize avalanche effect while maintaining computational efficiency ([see official documentation](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)). Multiple rotations ensure that changes in any input bit affect multiple output bit positions.

- For example
    - `Sigma0(5) = 0x40302001` demonstrates how rotations by 2, 13, and 22 bits create distinct patterns
    - The XOR combination ensures that small input changes produce large output differences

Sigma0's purpose is to provide strong diffusion properties that are essential for cryptographic security, as described in the SHA-256 design principles.

In [370]:
def sigma_upper_0(x):
    """Calculate the Sigma0 value for a 32-bit integer."""
    # The np.uint32() constructor ensures inputs are treated as unsigned 32-bit integers. 
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    # Mask with 0xFFFFFFFF to ensure only the lower 32 bits are kept to handle negative inputs correctly.
    x = np.uint32(x & 0xFFFFFFFF)
    
    # Compute the bitwise XOR of three right rotated versions of the input value(x).
    Sigma0_output = np.uint32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))
    # Return the result.
    return Sigma0_output

### Test Sigma0 Function
This section tests the `Sigma0` function with various inputs to verify its correctness.

In [371]:
# This array holds the test cases for the sigma_upper_0 function - each test case is a tuple containing: the test label, the input arguments, and the expected result.
tests = [
    ("\nTEST 1: sigma_upper_0(5)", (5,), 1076368385), # Test 1: A number other than 0 or 1
    ("\nTEST 2: sigma_upper_0(0)", (0,), 0), # Test 2: A zero input
    ("\nTEST 3: sigma_upper_0(1)", (1,), 1074267136), # Test 3: Input of one
    ("\nTEST 4: sigma_upper_0(-1)", (-1,), 4294967295), # Test 4: A negative input

]
# look through and run each test
for label, args, expected_result in tests:
    # Print thr test title.
    print(label)
    # Run the test arguments.
    test_result = sigma_upper_0(*args)
    # Print the result of the tests.
    print("Result :", hex(int(test_result)), "Expected :", hex(expected_result), "Correct :", int(test_result) == expected_result)



TEST 1: sigma_upper_0(5)
Result : 0x40281401 Expected : 0x40281401 Correct : True

TEST 2: sigma_upper_0(0)
Result : 0x0 Expected : 0x0 Correct : True

TEST 3: sigma_upper_0(1)
Result : 0x40080400 Expected : 0x40080400 Correct : True

TEST 4: sigma_upper_0(-1)
Result : 0xffffffff Expected : 0xffffffff Correct : True


### Sigma1 Function 
$\Sigma_1(x) = \text{ROTR}^6(x) \oplus \text{ROTR}^{11}(x) \oplus \text{ROTR}^{25}(x)$

The `sigma_upper_1` function applies a series of bitwise operations to a single 32-bit integer, as used in the SHA-224 and SHA-256 algorithms. It computes the bitwise XOR of three right-rotated versions of the input value - one rotated by 6 bits, one by 11 bits, and one by 25 bits. This is implemented as `ROTR^6(x) ^ ROTR^11(x) ^ ROTR^25(x)`.

Sigma1's distinct rotation pattern serves a **different cryptographic role** than Sigma0. While `Sigma0` operates on working variable `a` in the T2 computation, `Sigma1` operates on working variable `e` in the T1 computation ([see official documentation](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)). The asymmetric rotation amounts (6, 11, 25) were specifically chosen to complement `Sigma0's` pattern and ensure different bits influence different parts of each compression round.

- For example
    - `Sigma1(5) = 0x14a08000` vs `Sigma0(5) = 0x40302001` - demonstrating different bit patterns
    - `Sigma1` focuses on the `e` register state, while `Sigma0` focuses on the `a` register state in SHA-256 rounds

Sigma1's purpose is to provide **asymmetric diffusion** that complements Sigma0, ensuring that both halves of the SHA-256 compression function contribute different mixing properties for maximum cryptographic strength.

In [372]:
def sigma_upper_1(x):
    """Calculate the Sigma1 value for a 32-bit integer."""
    # The np.uint32() constructor ensures inputs are treated as unsigned 32-bit integers. 
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    # Mask with 0xFFFFFFFF to ensure only the lower 32 bits are kept to handle negative inputs correctly.
    x = np.uint32(x & 0xFFFFFFFF)

    # Compute the bitwise XOR of three right-rotated versions of the input value(x).
    Sigma1_output = np.uint32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))
    # Return the result.
    return Sigma1_output

### Test Sigma1 Function
This section tests the `Sigma1` function with various inputs to verify its correctness.

In [373]:
# This array holds the test cases for the sigma_upper_1 function - each test case is a tuple containing: the test label, the input arguments, and the expected result.
tests = [
    ("\nTEST 1: sigma_upper_1(5)", (5,), 346030720), # Test 1: Input other than 0 or 1
    ("\nTEST 2: sigma_upper_1(0)", (0,), 0), # Test 2: A zero input
    ("\nTEST 3: sigma_upper_1(1)", (1,), 69206144), # Test 3: An input of one
    ("\nTEST 4: sigma_upper_1(-1)", (-1,), 4294967295), # Test 4: A negative input
]
# look through and run each test
for label, args, expected_result in tests:
    # Print the test titles.
    print(label)
    # Run the test arguments.
    test_result = sigma_upper_1(*args)
    # Print the test results.
    print("Result :", hex(int(test_result)), "Expected :", hex(expected_result), "Correct :", int(test_result) == expected_result)



TEST 1: sigma_upper_1(5)
Result : 0x14a00280 Expected : 0x14a00280 Correct : True

TEST 2: sigma_upper_1(0)
Result : 0x0 Expected : 0x0 Correct : True

TEST 3: sigma_upper_1(1)
Result : 0x4200080 Expected : 0x4200080 Correct : True

TEST 4: sigma_upper_1(-1)
Result : 0xffffffff Expected : 0xffffffff Correct : True


### The sigma0 Function 
$\sigma_0(x) = \text{ROTR}^7(x) \oplus \text{ROTR}^{18}(x) \oplus \text{SHR}^3(x)$

The `sigma0` function applies a series of bitwise operations to a single 32-bit integer, as used in the SHA-224 and SHA-256 algorithms. It computes the bitwise XOR of two right-rotated versions of the input value - one rotated right by 7 bits, one rotated right by 18 bits and the value shifted right by 3 bits. This is implemented as `ROTR^7(x) ^ ROTR^18(x) ^ SHR^3(x)`.

The sigma0 function originates from the need for **message schedule expansion** in SHA-256's preprocessing phase. Unlike the uppercase Sigma0 function which operates on working variables during compression rounds, sigma0 transforms message words during the expansion from 16 to 64 words ([see official documentation](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)). The combination of rotations (7, 18) and shift (3) ensures effective bit mixing while maintaining computational efficiency.

- For example
    - `sigma0(5) = 0xa008000` demonstrates how the rotation and shift operations create distinct bit patterns
    - The function helps expand the initial 16 message words into the full 64-word schedule needed for SHA-256 rounds

sigma0's purpose is to provide **message diffusion** during the preprocessing phase, ensuring that changes in early message words propagate through the expanded message schedule.

In [374]:
def sigma_lower_0(x):
    """Calculate the sigma0 value for a 32-bit integer."""
    # The np.uint32() constructor ensures inputs are treated as unsigned 32-bit integers. 
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    # Mask with 0xFFFFFFFF to ensure only the lower 32 bits are kept to handle negative inputs correctly.
    x = np.uint32(x & 0xFFFFFFFF)

    # Compute the bitwise XOR of two right-rotated versions of the input value and the right-shifted value.
    sigma0_output = np.uint32(rotr(x, 7) ^ rotr(x, 18) ^ (x >> 3))
    # Return the result.
    return sigma0_output

### Test sigma0 Function 
This section tests the `sigma0` function with various inputs to verify its correctness.

In [375]:
# This array holds the test cases for the sigma_lower_0 function - each test case is a tuple containing: the test label, the input arguments, and the expected result.
tests = [
    ("\nTEST 1: sigma_lower_0(5)", (5,), 167854080), # Test 1: Input other than 0 or 1
    ("\nTEST 2: sigma_lower_0(0)", (0,), 0), # Test 2: A zero input
    ("\nTEST 3: sigma_lower_0(1)", (1,), 33570816), # Test 3: An input of one
    ("\nTEST 4: sigma_lower_0(-1)", (-1,), 536870911), # Test 4: A negative input
]
# look through and run each test
for label, args, expected_result in tests:
    # Print the test titles.
    print(label)
    # Run the test arguments.
    test_result = sigma_lower_0(*args)
    # Print the test results.
    print("Result :", hex(int(test_result)), "Expected :", hex(expected_result), "Correct :", int(test_result) == expected_result)



TEST 1: sigma_lower_0(5)
Result : 0xa014000 Expected : 0xa014000 Correct : True

TEST 2: sigma_lower_0(0)
Result : 0x0 Expected : 0x0 Correct : True

TEST 3: sigma_lower_0(1)
Result : 0x2004000 Expected : 0x2004000 Correct : True

TEST 4: sigma_lower_0(-1)
Result : 0x1fffffff Expected : 0x1fffffff Correct : True


### The sigma1 Function 
$\sigma_1(x) = \text{ROTR}^{17}(x) \oplus \text{ROTR}^{19}(x) \oplus \text{SHR}^{10}(x)$

The `sigma1` function applies a series of bitwise operations to a single 32-bit integer, as used in the SHA-224 and SHA-256 algorithms. It computes the bitwise XOR of two right-rotated versions of the input value - one rotated right by 17 bits, one rotated right by 19 bits - and the value shifted right by 10 bits. This is implemented as `ROTR^17(x) ^ ROTR^19(x) ^ SHR^10(x)`.

The sigma1 function originates from the need for **message schedule expansion** in SHA-256's preprocessing phase, working alongside `sigma0` to transform message words during the expansion from 16 to 64 words ([see official documentation](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)). The combination of rotations (17, 19) and shift (10) provides different bit mixing patterns than `sigma0`, ensuring comprehensive diffusion throughout the message schedule.

- For example
    - `sigma1(5) = 0x22000` demonstrates how the rotation and shift operations create distinct bit patterns
    - The function complements signma0 in expanding the message schedule, with different bit positions affected

`sigma1's` purpose is to provide **complementary message diffusion** during the preprocessing phase, ensuring that changes in message words propagate through different bit positions than `sigma0` for maximum cryptographic strength.

In [376]:
def sigma_lower_1(x):
    """Calculate the sigma1 value for a 32-bit integer."""
    # The np.uint32() constructor ensures inputs are treated as unsigned 32-bit integers. 
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    # Mask with 0xFFFFFFFF to ensure only the lower 32 bits are kept to handle negative inputs correctly.
    x = np.uint32(x & 0xFFFFFFFF)

    # Compute the bitwise XOR of two right-rotated versions of the input value and the right-shifted value
    sigma1_output = np.uint32(rotr(x, 17) ^ rotr(x, 19) ^ (x >> 10))
    # Return the result.
    return sigma1_output

### Test sigma1 Function 
This section tests the `sigma1` function with various inputs to verify its correctness.

In [377]:
# This array holds the test cases for the sigma_lower_1 function - each test case is a tuple containing: the test label, the input arguments, and the expected result.
tests = [
    ("\nTEST 1: sigma_lower_1(5)", (5,), 139264), # Test 1: Input other than 0 or 1
    ("\nTEST 2: sigma_lower_1(0)", (0,), 0), # Test 2: A zero input
    ("\nTEST 3: sigma_lower_1(1)", (1,), 40960), # Test 3: An input of one
    ("\nTEST 4: sigma_lower_1(-1)", (-1,), 4194303), # Test 4: A negative input

]
# look through and run each test
for label, args, expected_result in tests:
    # Print the test titles.
    print(label)
    # Run the test arguments.
    test_result = sigma_lower_1(*args)
    # Print the test results.
    print("Result :", hex(int(test_result)), "Expected :", hex(expected_result), "Correct :", int(test_result) == expected_result)



TEST 1: sigma_lower_1(5)
Result : 0x22000 Expected : 0x22000 Correct : True

TEST 2: sigma_lower_1(0)
Result : 0x0 Expected : 0x0 Correct : True

TEST 3: sigma_lower_1(1)
Result : 0xa000 Expected : 0xa000 Correct : True

TEST 4: sigma_lower_1(-1)
Result : 0x3fffff Expected : 0x3fffff Correct : True


## Problem 2: Fractional Parts of Cube Roots 
<!-- This section computes the SHA‑256 constant words by generating the first 64 primes, taking their cube roots, extracting the fractional parts, scaling each fractional part by 232 and truncating to capture the first 32 binary bits, formatting those 32‑bit values as 8‑hex‑digit words, and finally comparing the resulting hex list with the authoritative constants from FIPS 180‑4. -->

This problem demonstrates how SHA-256's round constants are derived from mathematical principles rather than arbitrary values. By extracting the fractional parts of cube roots from the first 64 prime numbers, we generate the 64 constant words ($K_0$ through $K_63$) used in each round of the SHA-256 compression function.

`The process involves:`
1. **Prime Generation**: Finding the first 64 prime numbers using SymPy
2. **Cube Root Calculation**: Computing the cube root of each prime
3. **Fractional Extraction**: Isolating the fractional parts of these cube roots
4. **Bit Scaling**: Multiplying by $2^{32}$ to capture the first 32 binary digits
5. **Hexadecimal Formatting**: Converting to 8-digit hex words for comparison
6. **Validation**: Verifying against the official FIPS 180-4 constants

- For example
    - The 2nd prime (3) has cube root $\sqrt[3]{3} \approx 1.442249570307408...$
    - Its fractional part $0.442249570307408... \times 2^{32}$ gives the constant `0x71374491`
    - This becomes $K_1$ in the SHA-256 round function

### The primes(n) function 
<!-- The `sympy.prime` function ([see official documentation](https://docs.sympy.org/latest/modules/ntheory.html#sympy.ntheory.generate.prime)) returns the nth prime number, and `sympy.primerange` ([see official documentation](https://docs.sympy.org/latest/modules/ntheory.html#sympy.ntheory.generate.primerange)) generates all prime numbers in a given range. These functions are used together in this project to efficiently return a list of the first *`n`* prime numbers.
This approach ensures both clarity and extensibility in the code.

### The primes(n) function  -->

The `primes(n)` function efficiently generates the first *n* prime numbers by combining two SymPy functions:

- **`sympy.prime(n)`** ([see official documentation](https://docs.sympy.org/latest/modules/ntheory.html#sympy.ntheory.generate.prime)) - returns the nth prime number
- **`sympy.primerange(start, stop)`** ([see official documentation](https://docs.sympy.org/latest/modules/ntheory.html#sympy.ntheory.generate.primerange)) - generates all prime numbers in a given range

**Implementation Logic:**
1. First, `sp.prime(n)` finds the *nth* prime number (example: `prime(10) = 29`)
2. Then, `sp.primerange(sp.prime(n) + 1)` generates all primes from 2 up to (but not including) the nth prime + 1
3. This ensures we get exactly the first *n* primes: `[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]`

`For Example:`
- `primes(5)` returns `[2, 3, 5, 7, 11]` (the first 5 primes)
- `primes(64)` returns the first 64 primes needed for SHA-256 constants

This approach ensures both **computational efficiency** (leveraging SymPy's optimised prime generation) and **code clarity** (readable single-line implementation).

In [378]:
def primes(n):
    """ Return a list of the first n prime numbers."""
    # Use sympy.prime to find the nth prime
    # See: https://docs.sympy.org/latest/modules/ntheory.html#sympy.ntheory.generate.prime
    # Use sympy.primerange to generate all primes up to and including the nth prime
    # See: https://docs.sympy.org/latest/modules/ntheory.html#sympy.ntheory.generate.primerange
    return list(sp.primerange(sp.prime(n) + 1))

### Calculate Cube Roots of the First 64 Primes
<!-- Generate the first 64 prime numbers using the `primes(n)` function. Next, calculate the cube root of each prime using NumPy's `cbrt` function ([see official documentation](https://numpy.org/devdocs/reference/generated/numpy.cbrt.html)), which efficiently applies the cube root operation to every element in the input array. This vectorised approach ensures accuracy and performance.

### Calculate Cube Roots of the First 64 Primes -->

This step generates the mathematical foundation for SHA-256's round constants by computing cube roots of the first 64 prime numbers.

`Process:`
1. **Prime Generation**: Use the `primes(64)` function to obtain the first 64 prime numbers: [2, 3, 5, 7, 11, 13, ...]
2. **Vectorised Computation**: Apply NumPy's `cbrt` function ([see official documentation](https://numpy.org/devdocs/reference/generated/numpy.cbrt.html)) to efficiently calculate cube roots for all primes simultaneously
3. **Mathematical Foundation**: These cube roots provide the irrational numbers needed for cryptographic constants

`Why Cube Roots?`
- Cube roots of primes yield irrational numbers with no discernible patterns
- The fractional parts of these irrationals become the SHA-256 round constants $K_0$ through $K_{63}$

`Implementation Benefits:`
- **Vectorised approach** - NumPy processes all `64` primes in a single operation
- **High precision** - Maintains accuracy needed for cryptographic applications  
- **Performance** - Efficient computation compared to iterative methods

`Example:`
- $\sqrt[3]{3} \approx 1.442249570307408...$ becomes the basis for constant  $K_1$ = `0x71374491`.


In [379]:
# Get the first 64 prime numbers.
primes_list = primes(64)  
# A list to hold the cube roots of the primes.
cube_roots = []
# Calculate the cube root of each prime number and store it in the list.
# np.cbrt computes the cube root of each element in the input array.
# See: https://numpy.org/devdocs/reference/generated/numpy.cbrt.html
cube_roots = np.cbrt(primes_list)
# Print the cube roots of the first 64 prime numbers.
print(cube_roots)

[1.25992105 1.44224957 1.70997595 1.91293118 2.22398009 2.35133469
 2.57128159 2.66840165 2.84386698 3.07231683 3.14138065 3.33222185
 3.44821724 3.50339806 3.60882608 3.75628575 3.89299642 3.93649718
 4.0615481  4.14081775 4.1793392  4.29084043 4.36207067 4.4647451
 4.59470089 4.65700951 4.68754815 4.7474594  4.77685618 4.83458813
 5.0265257  5.07875308 5.15513674 5.18010147 5.30145919 5.32507402
 5.39469071 5.46255557 5.50687845 5.57205466 5.63574079 5.65665283
 5.75896522 5.77899657 5.81864787 5.83827246 5.95334181 6.06412699
 6.1001702  6.11803317 6.15344949 6.20582179 6.22308425 6.30799355
 6.35786118 6.40695858 6.45531481 6.47127363 6.51868392 6.54991162
 6.56541443 6.6418522  6.74599671 6.77516895]


<!-- ### Extract fractional parts of the cube roots 
The NumPy `modf()` ([see official documentation](https://numpy.org/doc/stable/reference/generated/numpy.modf.html)) function returns the fractional and integral parts of an input array. This function is used for its speed and efficiency in separating the components. -->

### Extract Fractional Parts of the Cube Roots

This step isolates the fractional components of the cube roots, which contain the irrational decimal sequences needed for SHA-256's round constants.

**The NumPy `modf()`** function ([see official documentation](https://numpy.org/doc/stable/reference/generated/numpy.modf.html)) efficiently separates each floating-point number into its fractional and integral parts, returning both as arrays.

`Why Extract Fractional Parts?`
- **Integer parts** (1, 1, 1, ...) are predictable and cryptographically useless
- **Fractional parts** (0.442..., 0.587..., 0.709...) contain the irrational sequences
- These irrational decimals provide the numbers for SHA-256

`Example:`
For $\sqrt[3]{3} \approx 1.442249570307408...$
- **Integer part**: `1` (discarded)
- **Fractional part**: `0.442249570307408...` (used for constant generation)

`Implementation Benefits:`
- **Vectorised operation** - processes all 64 cube roots simultaneously
- **High precision** - maintains full floating-point accuracy
- **Efficiency** - single function call instead of manual separation

The extracted fractional parts become the foundation for deriving the 64 cryptographic constants $K_0$ through $K_{63}$ used in SHA-256 compression rounds.

In [380]:
# np.modf() function returns two tuples - the fractional and integral parts of the input array.
# See: https://numpy.org/doc/stable/reference/generated/numpy.modf.html
fractional, integer = np.modf(cube_roots)
# Print only the fractional parts of the cube roots.
print(fractional)

[0.25992105 0.44224957 0.70997595 0.91293118 0.22398009 0.35133469
 0.57128159 0.66840165 0.84386698 0.07231683 0.14138065 0.33222185
 0.44821724 0.50339806 0.60882608 0.75628575 0.89299642 0.93649718
 0.0615481  0.14081775 0.1793392  0.29084043 0.36207067 0.4647451
 0.59470089 0.65700951 0.68754815 0.7474594  0.77685618 0.83458813
 0.0265257  0.07875308 0.15513674 0.18010147 0.30145919 0.32507402
 0.39469071 0.46255557 0.50687845 0.57205466 0.63574079 0.65665283
 0.75896522 0.77899657 0.81864787 0.83827246 0.95334181 0.06412699
 0.1001702  0.11803317 0.15344949 0.20582179 0.22308425 0.30799355
 0.35786118 0.40695858 0.45531481 0.47127363 0.51868392 0.54991162
 0.56541443 0.6418522  0.74599671 0.77516895]


<!-- ### Extract first thirty-two bits of the fractional part
Shift the fractional part of the cube roots 32 bits in front of decimal point to bring the first 32 binary digits into the integer part, then convert it to an integer. -->

### Extract First Thirty-Two Bits of the Fractional Part

This step captures the first 32 binary digits from each fractional part by mathematically shifting the decimal point, effectively converting the leading fractional bits into integers for SHA-256 constant generation.

`Process:`
1. **Bit Shifting**: Multiply each fractional part by $2^{32}$ (4,294,967,296)
2. **Decimal Point Movement**: This moves the decimal point 32 positions right
3. **Integer Conversion**: Truncate to capture only the first 32 significant bits

`Why 32 Bits?`
- SHA-256 constants are exactly **32-bit words**
- Each fractional part contains infinite precision, but only the first 32 bits are needed
- This provides sufficient unpredictability while matching the algorithm's word size

` Example:`
For $\sqrt[3]{3} \approx 1.442249570307408...$:
- **Fractional part**: `0.442249570307408...`
- **After × 2³²**: `1899761150.45...` 
- **Integer conversion**: `1899761150` -> `0x71374491`
- **Result**: SHA-256 constant $K_1$

`Implementation Benefits:`
- **Bit-precise extraction** - Captures exactly the bits needed for cryptographic constants
- **Vectorised processing** - Handles all 64 fractional parts efficiently

The resulting 32-bit integers become the foundation for the hexadecimal SHA-256 round constants used in the compression function.

In [381]:
# A list to hold the first 32 bits of the fractional parts of the cube roots.
bits = []
# Loop through each fractional part of the cube roots.
for number in fractional:
    # Shift the fractional part 32 bits to the left to bring the digits into the integer part.
    shifted = number * (2 ** 32)
    # Convert the result to an integer and append it to the bits list.
    bits.append(int(shifted))

# Print the integer representation of the first 32 bits of the fractional parts of the cube roots.
print(bits)


[1116352408, 1899447441, 3049323471, 3921009573, 961987163, 1508970993, 2453635748, 2870763221, 3624381080, 310598401, 607225278, 1426881987, 1925078388, 2162078206, 2614888103, 3248222580, 3835390401, 4022224774, 264347078, 604807628, 770255983, 1249150122, 1555081692, 1996064986, 2554220882, 2821834349, 2952996808, 3210313671, 3336571891, 3584528711, 113926993, 338241895, 666307205, 773529912, 1294757372, 1396182291, 1695183700, 1986661051, 2177026350, 2456956037, 2730485921, 2820302411, 3259730800, 3345764771, 3516065817, 3600352804, 4094571909, 275423344, 430227734, 506948616, 659060556, 883997877, 958139571, 1322822218, 1537002063, 1747873779, 1955562222, 2024104815, 2227730452, 2361852424, 2428436474, 2756734187, 3204031479, 3329325298]


### Display Result in Hexadecimal

This step converts the extracted 32-bit integers into the standard 8-character hexadecimal format required for SHA-256 round constants.

`Process:`
1. **Format Conversion**: Each 32-bit integer is converted to an 8-digit lowercase hexadecimal string
2. **Zero-Padding**: The `:08x` format ensures leading zeros are included for consistency
3. **Standard Representation**: Results match the official FIPS 180-4 constant format

`Implementation Details:`
- **`f"{bit:08x}"`** - Python f-string formatting with 8-digit hex specification
- **Lowercase format** - Matches the standard SHA-256 constant representation
- **Fixed width** - Ensures all constants have exactly 8 hex characters

`Example Transformation:`
For the constant derived from $\sqrt[3]{3}$:
- **32-bit integer**: `1899761150`
- **Hexadecimal**: `71374491`
- **Result**: SHA-256 round constant $K_1$ = `0x71374491`

`Why Hexadecimal Format?`
- **Standard compliance** - FIPS 180-4 specifies constants in hexadecimal
- **Readability** - Easier to verify against official documentation
- **Implementation** - Most SHA-256 code uses hex constants directly

The resulting hex strings are ready for comparison with the authoritative SHA-256 constants defined in the Secure Hash Standard.

In [382]:
# A list to hold the hexadecimal representation of the bits.
hex_bits = []
for bit in bits:
    # Format each integer as an 8 character hexadecimal string.
    hex_bits.append(f"{bit:08x}")

# Print the hexadecimal representation of the bits.
print(hex_bits)


['428a2f98', '71374491', 'b5c0fbcf', 'e9b5dba5', '3956c25b', '59f111f1', '923f82a4', 'ab1c5ed5', 'd807aa98', '12835b01', '243185be', '550c7dc3', '72be5d74', '80deb1fe', '9bdc06a7', 'c19bf174', 'e49b69c1', 'efbe4786', '0fc19dc6', '240ca1cc', '2de92c6f', '4a7484aa', '5cb0a9dc', '76f988da', '983e5152', 'a831c66d', 'b00327c8', 'bf597fc7', 'c6e00bf3', 'd5a79147', '06ca6351', '14292967', '27b70a85', '2e1b2138', '4d2c6dfc', '53380d13', '650a7354', '766a0abb', '81c2c92e', '92722c85', 'a2bfe8a1', 'a81a664b', 'c24b8b70', 'c76c51a3', 'd192e819', 'd6990624', 'f40e3585', '106aa070', '19a4c116', '1e376c08', '2748774c', '34b0bcb5', '391c0cb3', '4ed8aa4a', '5b9cca4f', '682e6ff3', '748f82ee', '78a5636f', '84c87814', '8cc70208', '90befffa', 'a4506ceb', 'bef9a3f7', 'c67178f2']


### Test results against Secure Hash Standad 
This section compares the computed hexadecimal bits against the hex list defined in the Secure Hash Standard (FIPS 180-4).

In [383]:
# The expected SHA-256 constants (from FIPS 180-4 4.2.2) in hexadecimal format for comparison.
# See: https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
# K0-K63: First 64 constants derived from cube roots of primes k0...k63
shs_hex_list = [
    "428a2f98", "71374491", "b5c0fbcf", "e9b5dba5", "3956c25b", "59f111f1", "923f82a4", "ab1c5ed5",
    "d807aa98", "12835b01", "243185be", "550c7dc3", "72be5d74", "80deb1fe", "9bdc06a7", "c19bf174",
    "e49b69c1", "efbe4786", "0fc19dc6", "240ca1cc", "2de92c6f", "4a7484aa", "5cb0a9dc", "76f988da",
    "983e5152", "a831c66d", "b00327c8", "bf597fc7", "c6e00bf3", "d5a79147", "06ca6351", "14292967",
    "27b70a85", "2e1b2138", "4d2c6dfc", "53380d13", "650a7354", "766a0abb", "81c2c92e", "92722c85",
    "a2bfe8a1", "a81a664b", "c24b8b70", "c76c51a3", "d192e819", "d6990624", "f40e3585", "106aa070",
    "19a4c116", "1e376c08", "2748774c", "34b0bcb5", "391c0cb3", "4ed8aa4a", "5b9cca4f", "682e6ff3",
    "748f82ee", "78a5636f", "84c87814", "8cc70208", "90befffa", "a4506ceb", "bef9a3f7", "c67178f2"
]

# Compare the computed hexadecimal values with the expected SHA-256 constants
print("=== SHA-256 Constant Validation ===")
print(f"Generated constants: {len(hex_bits)}")
print(f"Expected constants:  {len(shs_hex_list)}")
print("***********************************")

# Compare the two lists: hex_bits is the computed list, shs_hex_list is the expected list
# If the lists match, print success message
if hex_bits == shs_hex_list:
    # Print success message
    print("SUCCESS: All computed hexadecimal values match the official SHA-256 constants!")
# If they do not match, print mismatch message
else:
    print("MISMATCH: Computed values do not match expected SHA-256 constants.")

=== SHA-256 Constant Validation ===
Generated constants: 64
Expected constants:  64
***********************************
SUCCESS: All computed hexadecimal values match the official SHA-256 constants!


## Problem 3: Padding

This section implements SHA-256's message preprocessing through two key functions that handle padding and block visualisation according to the Secure Hash Standard (FIPS 180-4).

`block_parse(msg)` - Message Padding and Block Generation

- The `block_parse` function is a memory-efficient generator that follows the SHA-256 padding specification. It accepts a bytes object as input and yields each 512-bit (64-byte) block after applying the required padding sequence:

    - The generator design avoids loading large messages entirely into memory, making it suitable for processing messages of any size. ([See source](https://www.geeksforgeeks.org/python/generators-in-python/))
<!-- 1. **Message Length Calculation** - Computes the original message length in bits
2. **Initial Padding** - Appends a single '1' bit followed by seven '0' bits (0x80 byte)
3. **Zero Padding** - Adds zero bytes until the message length ≡ 56 (mod 64)
4. **Length Field** - Appends the original message length as a 64-bit big-endian integer
5. **Block Generation** - Yields successive 64-byte blocks ready for SHA-256 processing -->


`print_blocks(msg)` - Block Visualisation and Verification

- The `print_blocks` helper function provides visual inspection of the padding and block parsing process. It:

    - This function is essential for understanding how messages are transformed into the block structure required by SHA-256 compression rounds.
<!-- - **Displays Input** - Shows the original message being processed
- **Binary Representation** - Converts each 64-byte block into readable 8-bit binary format
- **Block Enumeration** - Numbers each block for easy identification and debugging
- **Padding Verification** - Allows visual confirmation that padding follows SHA-256 standards -->

<!-- 
## Implementation Benefits:
- **Memory Efficiency** - Generator pattern handles large messages without memory issues
- **Standards Compliance** - Implements exact FIPS 180-4 padding specification  
- **Visual Debugging** - Binary output helps verify correct padding implementation
- **Modular Design** - Separate functions for processing and visualization -->

Both functions work together to ensure proper message preprocessing while providing transparency into the SHA-256 padding mechanism.


In [384]:
def block_parse(msg):
    """Pad a bytes message for SHA-256 and yield 64‑byte blocks."""
    # Compute the message length in bits.
    # len(msg) returns the number of bytes in the message; multiply by 8 to convert to bits.
    # This value is required to be appended as a 64-bit integer at the end of the padded message.
    message_length = len(msg) * 8  

    # Append 0x80 (a single '1' bit then seven '0' bits) as the required '1' bit padding.
    # This marks the start of the padding after the original message bytes.
    padded_message = msg + b'\x80'
    
    # Compute how many zero bytes (0x00) we must add so that the total length is congruentto 56 modulo 64. 
    # The reason for 56 is to leave exactly 8 bytes (64 bits)
    # At the end of the last block for storing the original message length.
    zero_padding = (56 - len(padded_message) % 64) % 64

    # Append the computed k zero bytes to padded_message in order to reach the 56 mod 64 boundary.
    padded_message += b'\x00' * zero_padding 

    # Append the original message length in bits as a 64-bit big-endian integer.
    # The SHA standard requires a 64-bit representation of the bit-length appended to the end of the padding. 
    # .to_bytes(8, 'big') produces exactly 8 bytes.
    # After this append, the total length of padded_message is a multiple of 64 bytes.
    padded_message += message_length.to_bytes(8, 'big')
    
    # Iterate over the padded_message and yield successive 64-byte chunks.
    # range(0, len(padded_message), 64) produces starting indices 0, 64, 128, ...
    # The slicing padded_message[i:i+64] extracts exactly 64 bytes for each block
    # See: https://www.w3schools.com/python/python_strings_slicing.asp
    # Using a generator (yield) avoids building an in-memory list of blocks, which is more memory-efficient for large messages.
    # See: https://www.geeksforgeeks.org/python/generators-in-python/
    for i in range(0, len(padded_message), 64):
        yield padded_message[i:i+64]


### Get the message length in bits 
`len(message) * 8`

This step calculates the original message length in bits, which is a critical component of SHA-256's padding specification according to FIPS 180-4.

`Process:`
- **Byte Count**: `len(message)` returns the number of bytes in the input message
- **Bit Conversion**: Multiply by 8 to convert bytes to bits (since 1 byte = 8 bits)
- **Length Storage**: This bit-length value will be appended as a 64-bit integer at the end of the padded message
<!-- 

One character is 8 bits. To get the lenght of the message in bits the length must be multiplied by 8.
len(message) returns the number of bytes (3). Multiply by 8 to get the length in bits (3 * 8 = 24). SHA padding and length fields use bit-length. -->

<!-- 
**Why Bits Instead of Bytes?**
- SHA-256 specification requires message length to be measured in **bits**, not bytes
- The final 64-bit length field in the padded message stores this bit-length value
- This ensures precise length encoding regardless of message size -->

This bit-length calculation is essential for the SHA-256 padding algorithm, ensuring that the original message size is preserved and encoded according to the cryptographic standard.

In [385]:
# For Example 
message = b"abc"
# Get the lenght of the message and multiply it by 8 to convert from bytes to bits.
message_length = len(message) * 8
# Print the message and its length in bits.
print(message)
print(message_length)

b'abc'
24


<!-- ### Append a byte with a single '1' bit followed by seven '0' bits
`message + b'\x80'`

`b'\x80'`is a bytes literal containing a single byte 0x80 (binary 10000000). In SHA padding this represents the required "1" bit followed by seven "0" bits. -->

### Append Initial Padding Bit
`message + b'\x80'`

This step implements the first requirement of SHA-256 padding by appending a mandatory '1' bit followed by seven '0' bits to mark the end of the original message data.

<!-- `Technical Details:` -->
- **`b'\x80'`** is a bytes literal containing the single byte `0x80` (hexadecimal)
- **Binary representation**: `10000000` - exactly one '1' bit followed by seven '0' bits

`Why This Step Is Required:`
- This ensures the padding is **always present**, even for messages that are multiples of 512 bits
- Without this marker, it would be impossible to distinguish between original message content and padding
- The additional seven '0' bits complete a full byte boundary for easier processing


The padded message will undergo further zero-padding and length field appending to reach the required 512-bit block boundary.


In [386]:
# For Example
# The original message in binary.
print("The original messag in binary: ",' '.join(f'{byte:08b}' for byte in message))  

# The message after padding with '1' bit and seven '0' bits at the end.
padded_message = message + b'\x80'
print("The message after padding with a byte", ' '.join(f'{byte:08b}' for byte in padded_message))   

The original messag in binary:  01100001 01100010 01100011
The message after padding with a byte 01100001 01100010 01100011 10000000


<!-- ### Compute number of zero bytes to reach 56 (mod 64)
`(56 - len(padded_message) % 64) % 64`

This calculates how many 0x00 bytes to append after the single 0x80 byte so the total length before the final 8‑byte length field is congruent to 56 modulo 64 (56 bytes into a 64‑byte block). SHA-256 requires the last 8 bytes of the final block to hold the message bit-length.


// ...existing code... -->
### Compute Number of Zero Bytes to Reach 56 (mod 64)
`(56 - len(padded_message) % 64) % 64`

This step calculates the exact number of zero bytes (`0x00`) needed to ensure the padded message length reaches the crucial 56-byte boundary within each 64-byte block, reserving the final 8 bytes for the message length field.

`Technical details`
- **`len(padded_message) % 64`** - Current position within the 64-byte block boundary
- **`56 - (current position)`** - Bytes needed to reach position 56
- **Outer `% 64`** - Handles the edge case when we're already at or past position 56

`Why 56 Bytes?`

- SHA-256 processes data in **512-bit (64-byte) blocks**
- The final **8 bytes** of each block must contain the original message length in bits
- This leaves exactly **56 bytes** available for message data and initial padding
- Formula: `64 - 8 = 56` (block size minus length field)

`Edge Case Handling:`

The double modulo operation ensures correct behavior in all scenarios:
- **Normal case**: If `len(padded_message) % 64 = 4`, then `(56 - 4) % 64 = 52` zero bytes needed
- **Edge case**: If `len(padded_message) % 64 = 60`, then `(56 - 60) % 64 = 60` zero bytes needed (forces new block)

This calculation ensures that after adding the zero padding, exactly 8 bytes remain in the current block for the length field, maintaining SHA-256's strict block structure requirements.


In [387]:
# Calculate the number of zero bytes to pad so that the length is 56 mod 64 bytes
zero_padding = (56 - len(padded_message) % 64) % 64
# Print the number of zero bytes to be added.
print(zero_padding)

52


<!-- ### Append k zero bytes 
`padded_message += b'\x00' * k`

Appends k zero bytes (each 0x00) to the bytes object padded_message.
Used here to ensure (message + 0x80 + zero_padding) is 56 bytes modulo 64 so the final 8‑byte length field can be appended and the full padded message length becomes a multiple of 64 bytes (512 bits), as required by SHA-256 padding.
-->

### Append Zero Padding Bytes
`padded_message += b'\x00' * zero_padding`

This step appends the calculated number of zero bytes (`0x00`) to the padded message, ensuring the total length reaches exactly the 56-byte boundary within each 64-byte block structure.

**Technical Implementation:**
- **`b'\x00'`** - Single zero byte (8 bits of zeros: `00000000`)
- **`* zero_padding`** - Python bytes multiplication creates the required number of zero bytes
- **`+=`** - Concatenates the zero bytes to the existing padded message

This zero padding ensures the padded message length is congruent to 56 modulo 64, creating the exact block structure required for SHA-256 processing.


In [388]:
# Append k zero bytes
# b'\x00' * zero_padding creates a bytes object with the required number of zero bytes.
# This is more efficient and clearer than using a loop to append each zero byte individually.
k_zero_padding = b'\x00' * zero_padding
# Print the zero bytes to be added.
# print(k_zero_padding)
print(f"Zero bytes padding in binary: {' '.join(f'{byte:08b}' for byte in k_zero_padding)}")


# Append the zero bytes to the padded message.
padded_message += k_zero_padding
# padded_message +=  b'\x00' * zero_padding
# Print the message after adding zero byte padding.
print("The message after adding zero bytes padding in binary:", ' '.join(f'{byte:08b}' for byte in padded_message))

Zero bytes padding in binary: 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000
The message after adding zero bytes padding in binary: 01100001 01100010 01100011 10000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 000000

### Append Original Message Length as 64-bit block
`message_length.to_bytes(8, 'big')`

This final step completes SHA-256 padding by appending the original message length in bits as a 64-bit big-endian integer, ensuring the total padded message length becomes a multiple of 512 bits.

`Technical Implementation:`
- **`message_length.to_bytes(8, 'big')`** - Converts the integer bit-length into exactly 8 bytes
- **`8`** - Creates a 64-bit (8-byte) representation as required by FIPS 180-4
- **`'big'`** - Uses big-endian byte ordering (most significant byte first)
- **`padded_message +=`** - Appends this length field to complete the padding

`Block Completion:`
This step ensures the final padded message length is exactly **64 bytes (512 bits)**, making it ready for SHA-256 block processing. The length field allows the algorithm to distinguish between original message content and padding during hash computation.

In [389]:
# Append the original message length as a 64-bit block 
# Convert message length (in bits) to 8 bytes using big-endian format
# This completes SHA-256 padding - total length must be multiple of 64 bytes
padded_message += message_length.to_bytes(8, 'big')

# Display the complete padded message
print(f"Final padded message length: {len(padded_message)} bytes")
print("Complete padded message:", ' '.join(f'{byte:08b}' for byte in padded_message))

Final padded message length: 64 bytes
Complete padded message: 01100001 01100010 01100011 10000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00011000


### Helper Function 
The  `print_blocks` helper fucntion prints each block in binary for review.

Prints the input message and then shows each 512‑bit (64‑byte) block produced by block_parse(msg) in binary per byte form. This helper function is used to verify SHA padding and block parsing.

In [390]:
def print_blocks(msg):
    """Print the binary representation of each 512-bit block."""
    # Display the original input message for reference
    print(f"Input: {msg}")
    # Parse the message into 512-bit blocks and print each block in binary format.
    # Use enumerate to get both the block data and its sequential number.
    for i, block in enumerate(block_parse(msg)):
        # Print block header starting from 1 for readability
        # Use end='' to keep the block number and binary data on the same line
        print(f"Block {i + 1}: ", end='') 
        # Convert each byte in the block to 8-bit binary representation
        # f'{byte:08b}' formats each byte as 8-digit binary.
        # ' '.join() separates each byte with a space for better readability.
        print(' '.join(f'{byte:08b}' for byte in block)) 
        
    # Add blank line after all blocks to seperate each output.
    print()

### Test Block Parse Function 
This section tests the `block_parse` function with various inputs to verify its correctness.The first step of SHA padding appends a single '1' bit and then zeros.
We represent that initial '1' bit plus seven zeros as the single byte `0x80` (binary `10000000`) and append it to the message.

Important: the message must be a `bytes` object (for example `b"abc"`).
If your message is a `str`, convert it first using `message.encode('utf-8')` before concatenating `b'\x80'`.

After appending `0x80` we will add zero bytes until the padded message length is congruent to 56 (mod 64),
and finally append the 64-bit big-endian length of the original message. The code cell below shows the immediate result after appending `0x80`.


In [391]:
# Test 1: Short message
print_blocks(b"abc")

# Test 2: Message that fits exactly in one block (55 bytes)
print_blocks(b"A" * 55)

# Test 3: Message that causes two blocks (more than 56 bytes)
print_blocks(b"B" * 60)

# Test 4: Empty message
print_blocks(b"")

# Test 5: Message of length exactly 64 bytes (should cause two blocks)
print_blocks(b"C" * 64)

Input: b'abc'
Block 1: 01100001 01100010 01100011 10000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00011000

Input: b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
Block 1: 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 01000001 0

## Problem 4: Hashes

This section implements the complete SHA-256 compression function according to **section 6.2.2 SHA-256 Hash Computation** from the Secure Hash Standard (FIPS 180-4). The implementation processes padded message blocks through the full 64-round compression algorithm to produce cryptographically secure hash digests.

`Key Components:`

1. **Message Block Preprocessing** - Converting 64-byte blocks into 16 32-bit big-endian words ([`get_message_blocks`](#preprocessing-stage-of-sha-256))
2. **Initial Hash Values** - Loading the eight SHA-256 constants ($H_0$ through $H_7$) ([`H_0`](#preprocessing-steps))
3. **Compression Function** - Implementing the 64-round SHA-256 algorithm with message schedule expansion ([`hash`](#the-hash-function))
4. **Digest Generation** - Converting the final 8-word state into a 32-byte hash output ([`H_to_digest`](#convert-the-internal-hash-state-into-the-final-32-byte-sha-256-digest))


`Process Overview:`
- **Input**: Current hash state (8 words) + 512-bit message block
- **Message Schedule**: Expand 16 input words to 64 words using [`sigma_lower_0`](#the-sigma0-function) and [`sigma_lower_1`](#the-sigma1-function) functions  
- **Compression Rounds**: 64 iterations using [`ch`](#ch-choose-function), [`maj`](#majority-function), [`sigma_upper_0`](#sigma0-function), and [`sigma_upper_1`](#sigma1-function) functions with round constants
- **Output**: Updated hash state ready for next block or final digest


`Example:`

For input `b"abc"`, the algorithm produces the standard SHA-256 digest:
`ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad`

This implementation demonstrates the mathematical precision and cryptographic rigor required for secure hash function design.


### Preprocessing stage of SHA-256 

This stage converts the padded message blocks from raw bytes into the 32-bit word format required for SHA-256 compression rounds. The `get_message_blocks` function transforms each 64-byte block into exactly 16 big-endian 32-bit words that can be processed by the hash algorithm.

`Process Overview:`
1. **Input**: Generator of 64-byte blocks from [`block_parse(msg)`](#problem-3-padding)
2. **Segmentation**: Each 64-byte block is split into 16 chunks of 4 bytes each
3. **Conversion**: Each 4-byte chunk is converted to a big-endian 32-bit integer
4. **Output**: List of blocks, where each block contains 16 integer words

`Example:`
For the message `b"abc"` after padding:
- **Input block**: 64 bytes starting with `0x61626380...`
- **First word**: `0x61626380` (combines bytes 'a', 'b', 'c', and padding byte 0x80)
- **Result**: 16 words ready for SHA-256 message schedule expansion

This preprocessing ensures that the SHA-256 compression function receives data in the exact format specified by FIPS 180-4.


In [392]:
def get_message_blocks(message):
    """Return a list of 512-bit blocks; each block is a list of 16 big-endian 32-bit words."""
    # Create an empty list to hold all blocks - each block is a list of 16 words
    blocks = []  
    # 'Message' is the result of call block_parse(msg) which yields 64-byte blocks and iterate over each block
    # We take each 64 byte block and split it into 16 pieces of 4 bytes so the algorithm can treat each piece as one 32‑bit number.
    for block_bytes in message:
        # Create a list to hold the 16 words for this current block
        words = []  
        # There are 16 words per 64-byte block
        for i in range(16):  
            # Take 4 bytes for each 32-bit word using slice notation
            # Example: i=0 -> [0:4], i=1 -> [4:8], i=2 -> [8:12] ....
            # This creates 16 chunks of 4 consecutive bytes from the 64-byte block
            chunk = block_bytes[i*4:(i+1)*4]  
            # Convert the 4 byte big-endian chunk into an int and append
            words.append(int.from_bytes(chunk, 'big'))  
            # Append the completed list of 16 words for this block to the blocks list
        blocks.append(words)  
     # Return the list of all blocks - each block is a list of 16 integers
    return blocks 

### Initial Hash Values 

This section defines the eight initial hash values ($H_0$ through $H_7$) required to start the SHA-256 algorithm, as specified in FIPS 180-4 section 5.3.3.

`Purpose:`
- Provides the starting point for SHA-256 hash computation
- These constants are derived from the fractional parts of square roots of the first 8 prime numbers
- Ensures consistent initialisation across all SHA-256 implementations

These values serve as the foundation for all SHA-256 hash computations, providing cryptographically secure starting conditions for the compression function.

In [393]:
# Initial hash values for SHA-256 section 6.2.2)
H_0_words = [
    np.uint32(0x6a09e667),
    np.uint32(0xbb67ae85),
    np.uint32(0x3c6ef372),
    np.uint32(0xa54ff53a),
    np.uint32(0x510e527f),
    np.uint32(0x9b05688c),
    np.uint32(0x1f83d9ab),
    np.uint32(0x5be0cd19),
]

# Function to return a copy of the initial hash values for SHA-256
def H_0():
    # Return a copy of the initial hash values list to prevent modification of the original.
    return H_0_words.copy()


In [394]:
# SHA-256 constants from FIPS 180-4 4.2.2 in hexadecimal format
# See: https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
# This list is used in the SHA-256 compression function to add to the message schedule and working variables.
K = [np.uint32(int(x, 16)) for x in shs_hex_list]

### The Hash function 

This function implements the SHA‑256 compression step (section 6.2.2 of the Secure Hash Standard). Its job is to take the current internal hash state (the 8-word chaining value) and one 512‑bit message block, run the 64 SHA‑256 rounds, and return the updated 8‑word intermediate state.

In [395]:
def hash(current, block):
    """Compute the SHA-256 hash of the given message blocks."""
    # Write a function hash(current, block) that calculates the next hash value given the current hash value and the next message block according to section 6.2.2 SHA-256 Hash Computation on page 22 of the Secure Hash Standard.
    # process each 512-bit block
    # W stores the 64 32-bit words for the current message block
    W = []
    # PART 1 - Prepare the message schedule for the first 16 32-bit blocks 
    # Each 512-bit block is split into 16 big-endian 32-bit words to prepare for the message schedule.
    for t in range(16):
        # Each word is copied into the list 'W' and masked with `0xffffffff` to ensure they remain 32-bit integers.
        W.append(int(block[t]) & 0xffffffff)
    # Prepare the message schedule for the first 16 32-bit blocks 
    # For each t in rnage 16-64 expand the message into a 64-word schedule that mixes input bits to finish the message schedule.
    for t in range(16, 64):
        s1 = sigma_lower_1(W[t - 2])
        s0 = sigma_lower_0(W[t - 15])
        new_w = ((s1) + W[t - 7] + int(s0) + W[t - 16]) & 0xffffffff
        # Append each new word to W, ensuring it remains a 32-bit integer.
        W.append(new_w)

    # PART 2 - Initialise the eight working variables 
    # Copy the intermediate hash words into working variables defined in the SHA-256 specification.
    # Mask each working variable with `0xffffffff` to keep 32-bit behavior. These variables are then updated by each of the 64 rounds.    
    a = int(current[0]) & 0xffffffff
    b = int(current[1]) & 0xffffffff
    c = int(current[2]) & 0xffffffff
    d = int(current[3]) & 0xffffffff
    e = int(current[4]) & 0xffffffff
    f = int(current[5]) & 0xffffffff
    g = int(current[6]) & 0xffffffff
    h = int(current[7]) & 0xffffffff

    # PART 3 - Shuffle the values 
    # For each round in the 64 rounds of SHA-256, update the working variables using the SHA-256 functions and constants.
    for t in range(64):
        T1 = (int(h) + (sigma_upper_1(e)) + int(ch(e, f, g)) + int(K[t]) + int(W[t])) & 0xffffffff
        T2 = (int(sigma_upper_0(a)) + int(maj(a, b, c))) & 0xffffffff

        h = g
        g = f
        f = e
        e = (d + T1) & 0xffffffff # Mask each addition with `0xffffffff` to model the 32-bit overflow.
        d = c
        c = b
        b = a
        a = (T1 + T2) & 0xffffffff # Mask each addition with `0xffffffff` to model the 32-bit overflow.

    # PART 4 - Compute the intermediate hash value
    # After 64 rounds, the working variables are added back into H:
    # Each variable is converted to np.uint32 to keep 32-bit representation.
    H =  [
        (a + current[0]) & 0xffffffff, 
        (b + current[1]) & 0xffffffff, 
        (c + current[2]) & 0xffffffff, 
        (d + current[3]) & 0xffffffff, 
        (e + current[4]) & 0xffffffff, 
        (f + current[5]) & 0xffffffff, 
        (g + current[6]) & 0xffffffff, 
        (h + current[7]) & 0xffffffff, 
    ]

    # return digest bytes (32 bytes)
    return H

### Convert Internal Hash State to Final 32-byte SHA-256 Digest
`H_to_digest(H)`

This function transforms SHA-256's internal 8-word hash state into the standard 32-byte digest format required for final hash output and comparison operations.

`Implementation:`
- **`int(word).to_bytes(4, 'big')`** - Converts each 32-bit word to 4-byte big-endian representation
- **`b''.join(...)`** - Concatenates the eight 4-byte words into a single 32-byte digest
- **Big-endian format** - Matches SHA-256 specification and ensures cross-platform consistency

`Purpose:`
This conversion bridges the gap between SHA-256's internal computational format and the standard hash digest format used for verification and storage. It produces the recognizable 64-character hexadecimal hash string when converted to hex.

`Result:` 8 words (256 bits) -> 32 bytes (256 bits) in standard digest format

In [396]:
def H_to_digest(H):
    """Convert list of 8 words into 32-byte digest bytes."""
    # Cast each word to an int then convert to 4 bytes big-endian.
    # int(word).to_bytes(4, 'big') : 4-byte big-endian representation of the 32-bit word.
    # b''.join(...) : concatenates the 8 four-byte words into the final 32-byte digest.
    return b''.join(int(word).to_bytes(4, 'big') for word in H)


<!-- ### Preprocessing steps 

- `H = H_0()` - load initial hash constants from a function to just call a simple H instead of a heax value evry time.
- `N = len(block)` - number of 512-bit blocks to process.
- `for i in range(N)` - process each block in order. -->


### Message Preprocessing for SHA-256

Convert the input message into the block structure required for SHA-256 compression.

`Steps:`
- **Pad and parse** the input message into 512-bit blocks (`b"abc"`)
- **Convert blocks** to lists of 16 big-endian 32-bit words as required by SHA-256
- Display the resulting message block structure

The output shows one block containing 16 words ready for SHA-256 processing.

In [397]:
# Preprocessing steps.
# Message as a bytes object. 
msg = (b"abc")
# Initialise the hash state to the SHA-256 initial constants.
H = H_0()
# Pad and parse the message to get the padded message as a generator of 512-bit blocks.
message = block_parse(msg)
# Convert the padded message into 512-bit blocks, each represented as 16 big-endian 32-bit words as required by SHA-256.
message_blocks = get_message_blocks(message)

# print the message blocks information
print("Words per block:", len(message_blocks[0]))         
print("The message bocks: ", message_blocks)           

Words per block: 16
The message bocks:  [[1633837952, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24]]


### Initial Message Schedule Setup

This section demonstrates the complete preprocessing pipeline required before SHA-256 compression rounds, transforming the input message into the format needed for hash computation.

`Message Schedule Initialization:`
- **Extract First Block** - Take `block0` (first 512-bit block with 16 words)
- **32-bit Masking** - Apply `0xFFFFFFFF` mask to ensure unsigned 32-bit behavior
- **W Array Setup** - Create the initial 16-word message schedule `W[0..15]`

`Purpose:`
These preprocessing steps transform the input message into the exact format required by the SHA-256 compression function. The resulting `W` array contains the first 16 words that will be expanded to 64 words during the compression rounds.

In [398]:
# Take the first 512-bit message block (contains 16 32-bit words)
block0 = message_blocks[0]
# Mask each word to 32 bits to ensure unsigned 32-bit behavior
W = [w & 0xFFFFFFFF for w in block0]
# Print the first 16 words in 8-digit hexadecimal form for inspection
print("W[0..15] (hex):", [f"{w:08x}" for w in W])

W[0..15] (hex): ['61626380', '00000000', '00000000', '00000000', '00000000', '00000000', '00000000', '00000000', '00000000', '00000000', '00000000', '00000000', '00000000', '00000000', '00000000', '00000018']


### Message Schedule Expansion (W[16] to W[63])

Expands the initial 16 message words into a full 64-word schedule using SHA-256's message expansion algorithm.

For each word `t` from 16 to 63 these steps are completed:

`Process:`
- **`sigma_lower_1(W[t-2])`** - Apply sigma1 shift to create non-linear bit mixing from recent word
- **`sigma_lower_0(W[t-15])`** - Apply sigma0 shift to diffuse bits from distant word  
- **Addition** - Combine all four components (`s1 + W[t - 7] + s0 + W[t - 16]`) using modular arithmetic to prevent simple bit pattern analysis
- **32-bit wraparound** - `& 0xFFFFFFFF` ensures overflow wraps around, maintaining word size
- **Result** - Each new word mixes bits from multiple previous positions, maximising avalanche effect

`Purpose:`
This expansion ensures that each of the 64 SHA-256 compression rounds has a unique word derived from the original message, providing extensive bit diffusion throughout the hash computation.

In [399]:
for t in range(16, 64):
    # Compute small-sigma1 on W[t-2] (bit shifts)
    s1 = sigma_lower_1(W[t - 2])
    # Compute small-sigma0 on W[t-15] (bit shifts)
    s0 = sigma_lower_0(W[t - 15])
    # Compute the new word and ensure it is a 32-bit integer
    new_word = (s1 + W[t - 7] + s0 + W[t - 16]) & 0xFFFFFFFF
    # Append the new word to the message schedule W
    W.append(new_word)

# Print a few expanded words in hex for inspection
print("Expanded words of W[16..19] in hex:", [f"{W[i]:08x}" for i in range(16, 20)])
# Confirm W has been expanded to 64 words
print("Total W length:", len(W))

Expanded words of W[16..19] in hex: ['61626380', '000f0000', '7da86405', '600003c6']
Total W length: 64


### Initialise Working Variables (a-h) from Current Hash State

Initialise a..h by copying the eight 32‑bit words from the current hash state H (masked to 32 bits) so the compression function operates on local working variables that are updated across the 64 rounds.

`Process:`
- **Copy H[0-7] -> a-h** - Creates local variables for round-by-round updates
- **32-bit masking** - `& 0xFFFFFFFF` ensures proper unsigned 32-bit behavior
- **Working variable setup** - These 8 variables will be transformed through 64 compression rounds

`Purpose:`
Working variables allow the compression function to modify temporary values while preserving the original hash state until all rounds complete.

In [400]:
# Copy intermediate hash words into working variables a..h, ensuring 32-bit wraparound
a = H[0] & 0xFFFFFFFF  
b = H[1] & 0xFFFFFFFF  
c = H[2] & 0xFFFFFFFF 
d = H[3] & 0xFFFFFFFF 
e = H[4] & 0xFFFFFFFF  
f = H[5] & 0xFFFFFFFF 
g = H[6] & 0xFFFFFFFF 
h = H[7] & 0xFFFFFFFF 

# Print the initial working variables in hexadecimal for quick inspection
print("Initial values (a..h) before compression begins: ", [hex(v) for v in (a, b, c, d, e, f, g, h)])

Initial values (a..h) before compression begins:  ['0x6a09e667', '0xbb67ae85', '0x3c6ef372', '0xa54ff53a', '0x510e527f', '0x9b05688c', '0x1f83d9ab', '0x5be0cd19']


### Single SHA-256 Compression Round Demonstration

Shows one complete SHA-256 compression round (t=0) with T1/T2 computation and working variable updates.

`T1 Computation:`
$T_1 = (h + \Sigma_1(e) + \text{Ch}(e,f,g) + K_0 + W_0) \bmod 2^{32}$
- Combines current hash state, round constant, message word, and SHA-256 functions

`T2 Computation:`
$T_2 = (\Sigma_0(a) + \text{Maj}(a,b,c)) \bmod 2^{32}$
- Applies sigma and majority functions to provide additional mixing

`Variable Updates:`
- **Rotation** - Most variables shift down ($h \to g$, $g \to f$, $f \to e$) to propagate information through the state
- **Special updates** - Only $e = (d + T_1)$ and $a = (T_1 + T_2)$ get new computed values to inject round-specific transformations
- **Why this pattern** - The rotation spreads changes across all variables while the two special updates introduce new entropy from the current round
- **32-bit masking** - `& 0xFFFFFFFF` ensures proper modular arithmetic and prevents integer overflow


`Purpose:`
This single round demonstrates the core SHA-256 transformation that will be repeated 64 times to fully process one message block.

In [401]:
t = 0
# Compute T1 and T2 cast results to Python int to avoid numpy scalar overflow,
# Then mask to 32 bits to model 32-bit wraparound)
T1 = (int(h) + int(sigma_upper_1(e)) + int(ch(e, f, g)) + int(block0[t]) + int(W[t])) & 0xFFFFFFFF
T2 = (int(sigma_upper_0(a)) + int(maj(a, b, c))) & 0xFFFFFFFF

# Shuffle variables as in SHA-256 round:
h = g
g = f
f = e
e = (d + T1) & 0xFFFFFFFF
d = c
c = b
b = a
a = (T1 + T2) & 0xFFFFFFFF

# Print the results 
print(f"Round {t}: T1={T1:08x}, T2={T2:08x}")
print("Updated working variables (a..h) after round 0:", [f"{v:08x}" for v in (a,b,c,d,e,f,g,h)])

Round 0: T1=73b284d0, T2=08909ae5
Updated working variables (a..h) after round 0: ['7c431fb5', '6a09e667', 'bb67ae85', '3c6ef372', '19027a0a', '510e527f', '9b05688c', '1f83d9ab']


### Test Hash
This section performs an end-to-end test of the SHA-256 implementation. It:
- uses the message b'abc',
- applies padding and parses the message into 512-bit blocks,
- runs the compression function on each block to update the intermediate hash state,
- converts the final 8-word state to a 32-byte digest and prints the hex representation.

The known correct SHA-256 digest for "abc" is:
ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad

Compare the printed out

In [ ]:
# Process each 512-bit block through the SHA-256 compression function.
for block in message_blocks:
    # Update the intermediate hash state with this block.
    # Takes the current hash state and the next message block as input.
    H = hash(H, block)
# After processing all blocks convert the final 8-word state into the 32-byte digest.
digest_hex = H_to_digest(H).hex()
# Print the final SHA-256 digest for the whole message.
print("Final SHA-256 Digest:", digest_hex)

SHA-256 Digest: ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad


## Problem 5: Passwords

This section demonstrates a practical application of the SHA-256 implementation by performing a dictionary attack to reverse-engineer passwords from their hash values.

`Approach:`
- **Dictionary Attack** - Test common passwords against the target hashes
- **Hash Comparison** - Use our SHA-256 implementation to hash each candidate password
- **Pattern Matching** - Compare computed hashes with the target values to find matches

`Implementation:`
The solution uses a password dictionary file and systematic hash comparison to recover the original passwords from their SHA-256 digests.

In [403]:
# The following are the SHA-256 hashes of three common passwords that have been hashed using one pass of the SHA-256 algorithm. 
# As strings, they were encoded using UTF-8.
hashed_passwords_list = ["5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8", 
"873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34","b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342"]

### Password Dictionary Attack Function

`find_password(hashed_passwords, password_list)`

Implements a dictionary attack to recover original passwords from their SHA-256 hash values by systematically testing common password candidates.

`Overview:`
1. **Setup** - Convert target hashes to a set for efficient lookup and initialise results dictionary
2. **File Processing** - Read password candidates from the dictionary file line by line
3. **Hash Computation** - For each candidate password:
   - Encode to UTF-8 bytes
   - Apply SHA-256 padding and block parsing
   - Process through full SHA-256 compression 
   - Convert final state to hexadecimal digest
4. **Comparison** - Check if computed hash matches any target hash
5. **Early Termination** - Stop searching once all target passwords are found

`Input:`
- `hashed_passwords` - List of SHA-256 hash strings to crack
- `password_list` - Path to dictionary file containing password candidates

`Output:`
- Dictionary mapping each cracked hash to its corresponding plaintext password

This function demonstrates the practical vulnerability of weak passwords and validates the correctness of the complete SHA-256 implementation.

In [404]:
def find_password(hashed_passwords, password_list):
    # Convert the list of hashed passwords to a set for faster lookup of target hashes
    # Set to lowercase to ensure case-insensitive matching
    targets = set(h.lower() for h in hashed_passwords)
    # Dictionary to hold found passwords
    found = {}
    # Open the password list file
    file = open(password_list)
    # Read all lines from the file
    passwords = file.readlines()

    # Iterate over each password in the list
    for password in passwords:
        # Remove trailing spaces from the text to get the actual password 
        password = password.strip()

        # Encode the password as UTF-8 to get bytes
        msg = password.encode('utf-8')
        # Pad and parse the message into blocks to prepare for hashing
        message = block_parse(msg) # yields 64-byte blocks to be processed
        message_blocks = get_message_blocks(message) # convert each 64-byte block into a list of 16 big-endian 32-bit words

        # Start from initial chaining value
        H = H_0()

        # Process each 512-bit block through the SHA-256 compression function.
        for block_words in message_blocks:
            # Update the intermediate hash state with this block.
            H = hash(H, block_words)

        # Convert the 8-word internal state into a 32-byte digest and then to hex.
        digest_hex = H_to_digest(H).hex()

        # Check if the computed hash matches any of the target hashes
        if digest_hex in hashed_passwords and digest_hex not in found:
            # Store the found password in the dictionary
            found[digest_hex] = password
            # stop early if all passwords have been found 
            if len(found) == len(targets):
                break

    # return the dictionary of found passwords 
    return found

### Load list of passwords from file 

The snippet sets up a single target hash (a SHA-256 hex string), points to a file containing password candidates, reads the file into memory, and prints how many candidate passwords were loaded.
This is the setup phase before iterating over candidates, hashing them, and comparing the results to the target hash.

Why `.lower()` was used
- Hexadecimal digits for SHA‑256 (0–9, a–f) can be represented in upper or lower-case. Normalising the target string with `.lower()` avoids mismatches caused only by letter case.
- Example:
  - `"ABCDEF".lower()` = `"abcdef"`.
  - If you compare `"8D96..."` to `"8d96..."` without normalising, a direct string equality test may fail even though the hex values are identical.

Why `encoding="utf-8"` is used when opening the password file
- Text files are stored as bytes with an encoding. `encoding="utf-8"` tells Python how to convert those bytes into Python str characters. The given hashed passwords were also encoded using `utf-8` so the same encoding must be used for comparison.

 - Example:
    - The string `"naïve"` encoded as UTF‑8 is `b"na\xc3\xafve"`. If encoded as Latin‑1 it would be `b"na\xefve"`. These are different bytes = different SHA‑256 digests.

In [405]:
# Example usage of one hashed password 
hashed_password = "8d969eef6ecad3c29a3a629280e686cf0c3f5d5a86aff3ca12020c923adc6c92"
# Path to your password list file
password_list = "data/common_passwords_10000.txt"  
# Convert the target hash to lowercase to ensure case-insensitive matching
hashed_password = hashed_password.lower()

# Open the password list and read all lines - using UTF-8 encoding to handle various characters
file = open(password_list, "r", encoding="utf-8")
passwords = file.readlines()
# Print the number of password candidates loaded
print("Loaded", len(passwords), "password candidates.")

Loaded 10000 password candidates.


### Prepare the plain text password to be hashed 
`password = passwords[0].strip()` Removes leading and trailing whitespace characters from the string. That includes spaces, tabs, newlines (`\n`), carriage returns (`\r`), and similar Unicode whitespace.
    - Example:
      - `"  pass\n".strip()` = `"pass"`
  - Why we call .strip() here:
    - When reading lines from a text file using `.readlines()` or iterating the file, each line typically ends with a newline (`\n`). Calling `.strip()` removes that newline so the string represents the exact password characters that should be hashed.
    - If you dont iunclude strip, hashing `"password\n"` (bytes include the newline) will produce a different digest than hashing `"password"`.

`block_parse` applies SHA‑256 padding and splits the padded message into 64‑byte (512‑bit) blocks.

`get_message_blocks` converts each 64‑byte block into 16 big‑endian 32‑bit words; these are the inputs for the hash compression function.

In [406]:
# Strip trailing spaces from the first password to get the actual password
password = passwords[0].strip() 

# # Encode the password as UTF-8 to get bytes
# msg = password.encode('utf-8')
# Pad and parse the message into blocks to prepare for hashing
message = block_parse(msg) # yields 64-byte blocks
message_blocks = get_message_blocks(message) # list of 16-word lists

# Print the message blocks
print(message_blocks) 

[[1633837952, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24]]


### Hash the plain text password 

`H = H_0()` returns the SHA‑256 initial hash state: eight 32‑bit words (the constants defined by the standard).
  - These eight words are the starting "chaining value" for the compression rounds.

`for block_words in message_blocks:`
  - `message_blocks` is an iterable of 512‑bit blocks prepared for the compressor.
    - Each block is represented as 16 32‑bit big‑endian words.
    - Each iteration applies the compression function to update the intermediate state: H = hash(H, block_words)

`digest_hex = H_to_digest(H).hex()`
  - `H_to_digest(H)` converts the final internal 8‑word state into the 32 raw bytes of the SHA‑256 digest. This means concatenating each 32‑bit word in big‑endian order.
  - `.hex()` turns those 32 bytes into a 64‑character hexadecimal string which is the usual representation of a SHA‑256 hash.

In [407]:
# Start from initial hash value
H = H_0()  

# Process each 512-bit block through the SHA-256 compression function.
for block_words in message_blocks:
    # Update the intermediate hash state with this block.
    H = hash(H, block_words)  

# Convert the 8-word internal state into a 32-byte digest and then to hex.
digest_hex = H_to_digest(H).hex()
# Check if the computed hash matches the target hash
match = digest_hex == hashed_password

print("Password candidate:", password)
print("Digest:", digest_hex)
print("Is match:", match)

Password candidate: 123456
Digest: ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
Is match: False


### Execute Dictionary Attack to Recover Passwords

Runs the password hash function against a common password dictionary to find the original passwords.

`Process:`
- **Dictionary Source** - Uses `common_passwords_10000.txt` containing 10,000 common passwords
- **Execution Time** - Takes approximately 30 seconds to process the dictionary
- **Hash Matching** - Compares each candidate password's SHA-256 hash against target hashes
- **Results Display** - Shows each successfully cracked password with its corresponding hash

This demonstrates the vulnerability of weak passwords and validates the complete SHA-256 implementation by successfully reversing the cryptographic hashes.

In [ ]:
# The file containing common passwords to test against the hashed passwords
passwords_file = "data/common_passwords_10000.txt"
# Find the passwords corresponding to the given hashed passwords
result = find_password(hashed_passwords_list, passwords_file)

# Loop through all passwords in found dictionary containing the password and the hash. 
# Loop through each password and its corresponding hash in the result dictionary.
for hashed, password in result.items():
    # Print the found password with its hash
    print(f"Password found: {password} -> {hashed}")


Password found: password -> 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
Password found: cheese -> 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
Password found: P@ssw0rd -> b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342


### How the passwords were found

To decipher the paswords that were hashed with one pass of the SHA-256 algorithm, I used a dictionary attack (with common passwords). A dictionary attack is essentially systematically trying every word in a dictionary to find a match.([see reference](https://rublon.com/blog/brute-force-dictionary-attack-difference/))


In this case, the dictionary used was a list of 10,000 common passwords. The `find_passwords()` operates by:

1. Taking in a list of hashed passwords and a text file of common passwords in plain text.</br>
2. Stripping the passwords of any trailing spaces.</br>
3. Encoding the passwords to standard UTF-8.</br>
4. Padding and parsing the password to be hashed.</br>
5. Hashing the password.</br>
6. Comparing the target hash against the hash computed from the dictionary.</br>

In short, I found the passwords by gathering a list of 10,000 common passwords, hasing each individual password in the list, and finally comparing it to the target hashes, which resulted in finding all target hashes.


### Suggest ways in which the hashing of passwords could be improved to prevent the kind of attack you performed to find the passwords

1. **`Prepending a salt`** </br>
    To increase the strength of passwords, prepending a salt can be used. Prepending is when information is added to the beginning of a password or a string to increase the security ([see reference](https://www.swindx.com/glossary/prepending/)). Salting is the act of adding a series of random characters to a password before going through the hashing function ([see reference](https://www.okta.com/blog/identity-security/what-are-salted-passwords-and-password-hashing/)).

    
    - When a user is creating a password for the first time a random salt is generated and added to the beginning of the users password to make it more secure. The combination of the salt and original password is then hashed and stored.
        <!-- </br> -->
        - The salt and the password value are stored seperately in a database but used together for password validation when they need to be hashed again.
    </br>

    **`Example of prepending the Salt`** </br>
    - Password: cheese</br>
    Salt: f1nd1ngn3m0</br>
    Salted input: f1nd1ngn3mcheese</br>
    Hash (SHA-256): 7528ed35c6ebf7e4661a02fd98ab88d92ccf4e48a4b27338fcc194b90ae8855c</br>


    **`Why this works against the attack performed`**
    </br>
    - Prepending a unique salt to the common password `cheese` turns it into a distinct credential that is extremely unlikely to appear in a common-password lists. This prevents attackers from matching stored hashes against precomputed dictionaries and forces them to test guesses individually for each salt, greatly increasing the cost of an attack.


As you can see, with a few simple changes prepending is a quick and efficient method to strenghten a password.

## End